In [1]:
import numpy as np
import pandas as pd

import json
import os
from tqdm import tqdm
import pickle
import re

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# Generate all_ids

In [3]:
with open('../results_example/X_train.pkl', 'rb') as file:
    X_train = pickle.load(file)

In [6]:
X_train.shape

(35, 5377)

In [2]:
with open('./results/X_train.pkl', 'rb') as file:
    X_train = pickle.load(file)

In [21]:
for i in range(1,8):
    print(1+i*768)

769
1537
2305
3073
3841
4609
5377


In [22]:
pca = PCA(n_components=80)
X_description_pca = pca.fit_transform(X_train[:, 1:769])
pca = PCA(n_components=80)
X_inclusion_pca = pca.fit_transform(X_train[:, 769:1537])


In [31]:
X_train[:, 0]

array([-0.00548419, -0.00421741, -0.00513141, ..., -0.00566057,
       -0.00465036, -0.00529177])

In [29]:
np.concatenate((X_train[:, [0]], X_description_pca), axis=1)

array([[-5.48418630e-03,  2.07992998e+00, -1.09566611e+00, ...,
        -5.76867569e-02,  1.38180805e-01, -9.36282265e-02],
       [-4.21741350e-03,  1.04926631e+00, -1.02716205e+00, ...,
         4.64628456e-01, -1.10033571e-01, -4.57400238e-02],
       [-5.13141413e-03, -1.84473822e+00,  5.29840416e-01, ...,
        -4.16255969e-01, -3.05884870e-01, -9.06071340e-02],
       ...,
       [-5.66057239e-03,  8.19845076e-01, -4.64275200e-01, ...,
         2.63984794e-02, -1.70322387e-01,  3.30412524e-02],
       [-4.65036117e-03,  1.97439599e+00, -1.84794664e+00, ...,
         2.71659852e-01,  5.16461913e-02, -9.03792689e-02],
       [-5.29176512e-03, -5.11180168e+00,  6.83578652e-01, ...,
        -8.93148388e-02,  6.05417266e-02, -1.22867497e-01]])

In [1]:
from joblib import load

model = load('./results/model_rf.sav')

In [22]:
with open('./results/X_train_pca.pkl', 'rb') as file:
    X_train_pca = pickle.load(file)

In [25]:
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
import numpy as np

2024-08-13 11:13:23.981248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 11:13:23.993523: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 11:13:23.997218: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-13 11:13:24.007623: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-08-13 11:13:24.824933: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [26]:
model = load_model('./results/model_cnn.keras')

In [27]:
with open('./results/X_test.pkl', 'rb') as file:
    X_test = pickle.load(file)

with open('./results/y_test.pkl', 'rb') as file:
    y_test = pickle.load(file)


In [28]:
predictions = model.predict(X_test)

361/361 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step


In [30]:
with open('./results/X_train_pca.pkl', 'rb') as file:
    X_train_pca = pickle.load(file)

In [33]:
X_train_pca.reshape(-1, 351, 1).shape

(104201, 351, 1)

In [15]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 20,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [5]:
y_pred = model.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred)

In [6]:
mse_ridge

404.7429081997575

In [27]:
def combine_pickle_files(directory_path):
    df_list = []
    file_names = ['description2embedding.pkl',
                  'eligibility2embedding.pkl',
                  'treatment2embedding.pkl',
                  'disease2embedding.pkl',
                  'outcome2embedding.pkl']

    for file_name in file_names:
        file_path = os.path.join(directory_path, file_name)
        with open(file_path, "rb") as f:
            p = pd.DataFrame(pickle.load(f))\
                .set_index('nctId')
            df_list.append(p)

    combined_df = pd.concat(df_list, axis=1, join="inner")
    combined_df.reset_index(inplace=True)

    return combined_df

In [28]:
df = combine_pickle_files('results')

In [30]:
df.shape

(168559, 9)

In [32]:
df.dropna().shape

(168456, 9)

In [11]:
def separate_cols(combined_df):
    separated_df = pd.concat([combined_df[['nctId', 'enrollment']],
            combined_df['description_embedding'].apply(pd.Series),
            combined_df['inclusion_embedding'].apply(pd.Series),
            combined_df['exclusion_embedding'].apply(pd.Series),
            combined_df['treatment_embedding'].apply(pd.Series),
            combined_df['disease_embedding'].apply(pd.Series),
            combined_df['measures_embedding'].apply(pd.Series),
            combined_df['timeframes_embedding'].apply(pd.Series)], axis=1)
    
    return separated_df

In [10]:
with open('./results/disease2embedding.pkl', "rb") as f:
    data = pd.DataFrame(pickle.load(f))

In [11]:
data.head(2)

,nctId,disease_embedding
0,NCT03202394,"[0.14991804957389832, 0.22459396719932556, -0...."
1,NCT02138214,"[-0.020751827706893284, 0.2324144939581553, -0..."


In [4]:
data.head(2)

,nctId,measures_embedding,timeframes_embedding
0,NCT03202394,"[-0.04192572832107544, -0.11662426590919495, -...","[0.27893149852752686, -0.09838172048330307, -0..."
1,NCT02138214,"[0.0962123335339129, -0.03012945014052093, -0....","[0.17500807121541584, 0.024294625967741013, -0..."


In [36]:
with open('./data_example/X_train.pkl', 'rb') as file:
    # Load data from the file
    X_train = pickle.load(file)
    
with open('./data_example/y_train.pkl', 'rb') as file:
    # Load data from the file
    y_train = pickle.load(file)

ModuleNotFoundError: No module named 'pandas.core.indexes.numeric'

In [38]:
X_train.shape

(35, 5377)

In [ ]:
df = generate_train_test_incompleted_ids('./data_all/clinIDs_parsed.pkl')

In [ ]:
df

# Data Import

In [ ]:
def prepare_dataset(file_path):
    df = pd.read_pickle(file_path)

    X = pd.concat([df['enrollment'],
                   df['description_embedding'].apply(pd.Series),
                   df['inclusion_embedding'].apply(pd.Series),
                   df['exclusion_embedding'].apply(pd.Series),
                   df['treatment_embedding'].apply(pd.Series),
                   df['disease_embedding'].apply(pd.Series),
                   df['measures_embedding'].apply(pd.Series),
                   df['timeframes_embedding'].apply(pd.Series)], axis=1)
    y = df['durationMonths']

    return X, y

X_train, y_train = prepare_dataset('./data_example/train_df.pkl')
X_test, y_test = prepare_dataset('./data_example/test_df.pkl')
X_incompleted, y_incompleted = prepare_dataset('./data_example/incompleted_df.pkl')

In [ ]:
# standardize enrollment
scaler = StandardScaler()
X_train_enroll_scaled = scaler.fit_transform(X_train[['enrollment']])
X_train_scaled = np.concatenate([X_train_enroll_scaled, X_train.iloc[:, 1:].values], axis=1)

X_test_enroll_scaled = scaler.transform(X_test[['enrollment']])
X_test_scaled = np.concatenate([X_test_enroll_scaled, X_test.iloc[:, 1:].values], axis=1)

X_incompleted_enroll_scaled = scaler.transform(X_incompleted[['enrollment']])
X_incompleted_scaled = np.concatenate([X_incompleted_enroll_scaled, X_incompleted.iloc[:, 1:].values], axis=1)

# Linear Regression

In [ ]:
# modeling training
model_ridge = Ridge()
param_grid = {'alpha': [0.1, 1.0, 10.0]}

grid_search = GridSearchCV(
    estimator=model_ridge,
    param_grid=param_grid, cv=5,
    scoring='neg_mean_squared_error'
)
grid_search.fit(X_train_scaled, y_train)
best_model_ridge = grid_search.best_estimator_

# Evaluation
y_pred_ridge = best_model_ridge.predict(X_test_scaled)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)


# Random Forest

In [ ]:
# Initialize the model
model_rf = RandomForestRegressor(random_state=42)

# Tune the parameter
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_leaf': [1, 5, 10]
}

random_search = RandomizedSearchCV(
    estimator=model_rf,
    param_distributions=param_grid, 
    n_iter=10, cv=5,
    scoring='neg_mean_squared_error',
    random_state=42
)

random_search.fit(X_train_scaled, y_train)
best_model_rf = random_search.best_estimator_

# Evaluation
y_pred_rf = best_model_rf.predict(X_test_scaled)
mse_rf = mean_squared_error(y_test, y_pred_rf)

# XGBoost

In [ ]:
import xgboost as xgb
import shap

In [ ]:
# Initialize the XGBoost regressor
model_xgb = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Tune the parameter
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001]
}

random_search = RandomizedSearchCV(
    estimator=model_xgb,
    param_distributions=param_grid, 
    n_iter=10, cv=5,
    scoring='neg_mean_squared_error',
    random_state=42
)

random_search.fit(X_train_scaled, y_train)
best_model_xgb = random_search.best_estimator_

# Evaluation
y_pred_xgb = best_model_xgb.predict(X_test_scaled)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)

In [ ]:
# Create an explainer object using the XGBoost model
explainer = shap.Explainer(best_model_xgb)

# Compute SHAP values
shap_values = explainer(X_train_scaled)

In [ ]:
# Aggregate SHAP values for groups of features
sections = ['enrollment', 'description', 'inclusion', 'exclusion',
            'treatment', 'disease', 'measure', 'timeframe']

section_shap_values = {}

for i, section in enumerate(sections):
    if section == 'enrollment':
        section_shap_value = shap_values.values[:, 0]
    else:
        start_idx = (i-1) * 768 + 1
        end_idx = start_idx + 768
        section_shap_value = np.sum(shap_values.values[:, start_idx:end_idx], axis=1)
    
    section_shap_values[section] = section_shap_value

section_shap_values = pd.DataFrame(section_shap_values)

# FFNN

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Initialize the model
model_ffnn = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

# Compile the model
model_ffnn.compile(optimizer='adam', loss='mean_squared_error')

# Define callbacks (optional but recommended for saving the best model)
checkpoint = ModelCheckpoint('./results/ffnn_model.keras', monitor='val_loss',
                             save_best_only=True, mode='min', verbose=1)

# Train the model
model_ffnn.fit(X_train_scaled, y_train,
               epochs=50, batch_size=32,
               validation_data=(X_test_scaled, y_test),
               callbacks=[checkpoint])

# Load the best saved model
model_ffnn = load_model('./results/ffnn_model.keras')

# Predict on the test set
y_pred_ffnn = model_ffnn.predict(X_test_scaled)

# CNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Initialize the model
model_cnn = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])

# Compile the model
model_cnn.compile(optimizer='adam', loss='mean_squared_error')

# Define callbacks (optional)
checkpoint = ModelCheckpoint('./results/cnn_model.keras', monitor='val_loss',
                             save_best_only=True, mode='min', verbose=1)

# Train the model
model_cnn.fit(X_train_scaled, y_train,
              epochs=50, batch_size=32,
              validation_data=(X_test_scaled, y_test),
              callbacks=[checkpoint])

# Load the best saved model
model_cnn = load_model('./results/cnn_model.keras')

# Predict on the test set
y_pred_cnn = model_cnn.predict(X_test_scaled)

# LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# Initialize the model
model_lstm = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
    Dense(1)
])

# Compile the model
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Define callbacks (optional)
checkpoint = ModelCheckpoint('./results/lstm_model.keras', monitor='val_loss',
                             save_best_only=True, mode='min', verbose=1)

# Train the model
model_cnn.fit(X_train_scaled, y_train,
              epochs=50, batch_size=32,
              validation_data=(X_test_scaled, y_test),
              callbacks=[checkpoint])

# Load the best saved model
model_cnn = load_model('./results/lstm_model.keras')

# Predict on the test set
y_pred_cnn = model_cnn.predict(X_test_scaled)